# Exploring Numpy Array Broadcasting rules

Key idea = The Broadcasting Rule

> In order to broadcast, the size of the trailing axes for both arrays in an operation must either be the same size or one of them must be one.



see: 
 - https://numpy.org/doc/stable/user/basics.broadcasting.html
 - https://numpy.org/devdocs/user/theory.broadcasting.html


In [1]:
import numpy as np


In [126]:
X = np.array(np.mat(
"""
0 1 2;
1 1 1;
2 2 2;
3 3 3
"""))
print(X.shape)
X

(4, 3)


array([[0, 1, 2],
       [1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [127]:
v = np.array([10,11,12])
print(v.shape)
v

(3,)


array([10, 11, 12])

In [158]:
print('X.shape', X.shape)
print('v.shape', v.shape)
X + v

X.shape (4, 3)
v.shape (3,)


array([[10, 12, 14],
       [11, 12, 13],
       [12, 13, 14],
       [13, 14, 15]])

In [156]:
# alternatively make explicit 1
vv = np.reshape(v, (1,3))
print(vv.shape)
vv

(1, 3)


array([[10, 11, 12]])

In [157]:
print('X.shape', X.shape)
print('vv.shape', vv.shape)
X + vv

X.shape (4, 3)
vv.shape (1, 3)


array([[10, 12, 14],
       [11, 12, 13],
       [12, 13, 14],
       [13, 14, 15]])

In [204]:
Y = np.array(np.mat(
"""
5 5 5;
6 6 6;
7 7 7;
8 8 8;
9 9 9
"""))
Y

array([[5, 5, 5],
       [6, 6, 6],
       [7, 7, 7],
       [8, 8, 8],
       [9, 9, 9]])

## One for loop approach

In [205]:
diffs1 = np.zeros( (X.shape[0], Y.shape[0]) )

for i in range(X.shape[0]):
    x_test = X[i,:]
    # diffs1[i,:] = -1*np.sum(Y - x_test, axis=1)
    diffs1[i,:] = np.sqrt(np.sum(np.square(Y - x_test), axis=1))


diffs1

array([[ 7.07106781,  8.77496439, 10.48808848, 12.20655562, 13.92838828],
       [ 6.92820323,  8.66025404, 10.39230485, 12.12435565, 13.85640646],
       [ 5.19615242,  6.92820323,  8.66025404, 10.39230485, 12.12435565],
       [ 3.46410162,  5.19615242,  6.92820323,  8.66025404, 10.39230485]])

In [207]:
# check diffs1 against naive method (two for loops)
for i in range(X.shape[0]):
    for j in range(Y.shape[0]):
        expected_ij = np.sqrt(np.sum(np.square(X[i,:]-Y[j,:])))
        assert np.isclose(diffs1[i,j], expected_ij), f'mismatch at i={i}, j={j}'
print('All close')


All close


In [195]:
Y - x_test

array([[2, 2, 2],
       [3, 3, 3],
       [4, 4, 4],
       [5, 5, 5],
       [6, 6, 6]])

In [197]:
np.sqrt( np.sum(np.square(Y - x_test), axis=1) )

array([ 3.46410162,  5.19615242,  6.92820323,  8.66025404, 10.39230485])

In [192]:
dists_row = np.sum(np.square(Y - x_test), axis=1)
dists_row

array([ -12,  -27,  -48,  -75, -108])

## Zero for loops approach (memory inefficient)

In [135]:
# need to augment X so it has a trailing-1 dimension
XX = X[:, :, np.newaxis]
print(XX.shape)
# XX

(4, 3, 1)


In [148]:
## Another way to do the same thing w/o using np.newaxis
# XX = X.reshape(*X.shape, 1)
# print(XX.shape)
# XX

In [254]:
# compute the difference
print('XX.shape', XX.shape)
print('Y.T.shape', Y.T.shape)
print('(XX - Y.T).shape', (XX - Y.T).shape)
XX - Y.T

XX.shape (4, 3, 1)
Y.T.shape (3, 5)
(XX - Y.T).shape (4, 3, 5)


array([[[-5, -6, -7, -8, -9],
        [-4, -5, -6, -7, -8],
        [-3, -4, -5, -6, -7]],

       [[-4, -5, -6, -7, -8],
        [-4, -5, -6, -7, -8],
        [-4, -5, -6, -7, -8]],

       [[-3, -4, -5, -6, -7],
        [-3, -4, -5, -6, -7],
        [-3, -4, -5, -6, -7]],

       [[-2, -3, -4, -5, -6],
        [-2, -3, -4, -5, -6],
        [-2, -3, -4, -5, -6]]])

In [230]:
# reduce using sum along middle dimension
diffs0m = np.sqrt(np.sum(np.square(XX - Y.T), axis=1))
diffs0m

array([[ 7.07106781,  8.77496439, 10.48808848, 12.20655562, 13.92838828],
       [ 6.92820323,  8.66025404, 10.39230485, 12.12435565, 13.85640646],
       [ 5.19615242,  6.92820323,  8.66025404, 10.39230485, 12.12435565],
       [ 3.46410162,  5.19615242,  6.92820323,  8.66025404, 10.39230485]])

In [231]:
# check diffs0m against naive method (two for loops)
for i in range(X.shape[0]):
    for j in range(Y.shape[0]):
        expected_ij = np.sqrt(np.sum(np.square(X[i,:]-Y[j,:])))
        assert np.isclose(diffs0m[i,j], expected_ij), f'mismatch at i={i}, j={j}'
print('All close')

All close


In [258]:
# note this is very memory inefficient
# float64 or int64 take up 8 bytes (64 bits)
# and we have an intermediate result array of shape (4, 3, 5) 
# so this way would require allocating memory for intemrediate step:
print('Using this memory inefficient way in assignment would require')
print(8*(4*3*5), 'bytes')

Using this memory inefficient way in assignment would require
480 bytes


In [259]:
# float64 or int64 take up 8 bytes (64 bits)
# and we have an intermediate result array of shape (500, 3072, 5000) 
# so this way would require:
print('Using this memory inefficient way in assignment would require')
print(8*(500*3072*5000)/1024/1024/1024, 'GB')

Using this memory inefficient way in assignment would require
57.220458984375 GB


## Zero for loops approach (fast)

Rewrite using 

![rewrite trick](https://miro.medium.com/max/776/1*ZU5JN1yhEr6ga_N772gInA.png)

In [215]:
X

array([[0, 1, 2],
       [1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [224]:
XSQ = np.sum(np.square(X), axis=1)[:, np.newaxis]
XSQ

array([[ 5],
       [ 3],
       [12],
       [27]])

In [225]:
YSQ = np.sum(np.square(Y), axis=1)[np.newaxis,:]
YSQ

array([[ 75, 108, 147, 192, 243]])

In [236]:
XSQ + YSQ

array([[ 80, 113, 152, 197, 248],
       [ 78, 111, 150, 195, 246],
       [ 87, 120, 159, 204, 255],
       [102, 135, 174, 219, 270]])

In [238]:
diffs0 = np.sqrt(XSQ + YSQ - 2*X.dot(Y.T))

In [239]:
diffs0

array([[ 7.07106781,  8.77496439, 10.48808848, 12.20655562, 13.92838828],
       [ 6.92820323,  8.66025404, 10.39230485, 12.12435565, 13.85640646],
       [ 5.19615242,  6.92820323,  8.66025404, 10.39230485, 12.12435565],
       [ 3.46410162,  5.19615242,  6.92820323,  8.66025404, 10.39230485]])

In [240]:
# check diffs0 against naive method (two for loops)
for i in range(X.shape[0]):
    for j in range(Y.shape[0]):
        expected_ij = np.sqrt(np.sum(np.square(X[i,:]-Y[j,:])))
        assert np.isclose(diffs0[i,j], expected_ij), f'mismatch at i={i}, j={j}'
print('All close')

All close


In [260]:
x = np.arange(9)

In [268]:
splits = np.array_split(x, 4)
splits

[array([0, 1, 2]), array([3, 4]), array([5, 6]), array([7, 8])]

In [272]:
np.concatenate((splits[0], splits[2]))

array([0, 1, 2, 5, 6])

In [274]:
accuracies = np.zeros(3)
accuracies

array([0., 0., 0.])